# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

## Libraries and Dataset

In [ ]:
from google.colab import drive
import requests
import torchvision.transforms as transforms
from torch.autograd import Variable
import cv2
import os
from scipy.io import savemat
from scipy.io import loadmat

In [ ]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd '/content/drive/My Drive/CV/Homework/HW5/CSE527-HW5-Fall19'

/content/drive/My Drive/CV/Homework/HW5/CSE527-HW5-Fall19


In [ ]:
%%capture
!tar -xvf  'dataset/Test2/UCF101_images.tar' -C '/content/drive/My Drive/CV/Homework/HW5/CSE527-HW5-Fall19/dataset/Test2/img'

In [ ]:
filepath="http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar"
newfilepath="dataset/Test2/UCF101_images.tar"

file_url = filepath

print(file_url)
r = requests.get(file_url, stream = True)  

with open(newfilepath, "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
        if block:  
            file.write(block)
print(newfilepath)    

In [ ]:
import tarfile
tar = tarfile.open("dataset/Test2/UCF101_images.tar")

In [ ]:
%%time
tar.extractall()
tar.close()

In [ ]:
%%capture
# !tar -xkf  'dataset/Test2/UCF101_images.tar' -C '/content/drive/My Drive/CV/Homework/HW5/CSE527-HW5-Fall19/dataset/Test2/img'

In [ ]:
import tarfile
tar = tarfile.open("dataset/Test2/UCF101_images.tar", "r:")
tar.extractall()

In [ ]:

tar.close()

In [ ]:
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)
%%time
with open("annos/videos_labels_subsets.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
video_names=[]
labels=[]
data_use=[]
for c in content:
  temp=c.split('\t')
  video_names.append(temp[0])
  labels.append(temp[1])
  data_use.append(temp[2])

train_video_names=[video_names[i] for i in range(len(data_use)) if data_use[i] is '1' and int(labels[i]) in range(1, 102)]
train_video_labels=[int(labels[i])-1 for i in range(len(data_use)) if data_use[i] is '1' and int(labels[i]) in range(1, 102)]

test_video_names=[video_names[i] for i in range(len(data_use)) if data_use[i] is '2' and int(labels[i]) in range(1, 102)]
test_video_labels=[int(labels[i])-1 for i in range(len(data_use)) if data_use[i] is '2' and int(labels[i]) in range(1, 102)]

print("Number of training videos:", len(train_video_labels))
print("Number of testing videos:", len(test_video_labels))


Number of training videos: 9537
Number of testing videos: 3783
CPU times: user 24.6 ms, sys: 2.57 ms, total: 27.2 ms
Wall time: 948 ms


In [ ]:
%%time
import torch
import torch.nn as nn
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

#extract first fully connected layer
first_fully_connected_vgg16 = nn.Sequential(*list(vgg16.classifier.children())[:1])
vgg16.classifier=first_fully_connected_vgg16

#Set up GPU
device = torch.device("cuda:0")
vgg16.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 85.4MB/s]


CPU times: user 4.02 s, sys: 2.18 s, total: 6.2 s
Wall time: 13.5 s


In [ ]:
%%time
import os
import cv2
import torchvision.transforms as transforms
from torch.autograd import Variable
from scipy.io import savemat
from scipy.io import loadmat

def crop_2_224x224_and_form_tensors(img):
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  prep = transforms.Compose([ transforms.ToTensor(), normalize ])
  c=[]
  c.append(prep(img[0:224, 0:224]))  
  c.append(prep(img[32:256, 0:224]))
  c.append(prep(img[0:224, 32:256]))
  c.append(prep(img[32:256, 32:256]))
  c.append(prep(img[16:240, 16:240]))
  return c


def getClassBatch(start_vid,end_vid):
  videos=[]
  for index,vid_folder in enumerate(sorted(os.listdir("dataset/Test2/img/images/"))):
    if(index>=start_vid and index<end_vid):
      vid=[]
      for i in os.listdir('dataset/Test2/img/images/'+vid_folder):
        x=cv2.imread('dataset/Test2/img/images/'+vid_folder+'/'+i)
        vid.append(x)
      videos.append(vid)
      # print(index)
      # print(vid_folder)
    elif index==end_vid:
      print(vid_folder)
      break
  return videos

def save_vid_mat(videos,start_vid):
  all_ftrs=[]
  i=start_vid
  # print("Generating features for")
  for video in videos:
    # print("video", i+1, '- dataset/mat/'+video_names[i]+'.mat')
    mean_fetr=None
    video_ftrs=[]
    ftr_data={}
    for frame in video:
      crops=crop_2_224x224_and_form_tensors(frame)
      fetrs=[]
      for crop in crops:
        fetrs.append(vgg16(crop.unsqueeze(0).to(device)))
      fetrs=torch.stack(fetrs)
      mean_fetr=torch.mean(fetrs.cpu(), 0).detach().numpy()[0].tolist()
      video_ftrs.append(mean_fetr)
    ftr_data['Feature']=video_ftrs
    savemat('dataset/mat/'+video_names[i]+'.mat', ftr_data)
    i+=1
  print("video", i, '- dataset/mat/'+video_names[i-1]+'.mat')

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 26.7 µs


In [ ]:
start_vid=4500
end_vid=5000
videos=getClassBatch(start_vid,end_vid)
save_vid_mat(videos,start_vid)

v_005001
video 5000 - dataset/mat/v_005000.mat


---
---
## **Problem 1.** Feature extraction

In [ ]:
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)
%%time
with open("annos/videos_labels_subsets.txt") as f:
    content = f.readlines()
content = [x.strip() for x in content] 
video_names=[]
labels=[]
data_use=[]
for c in content:
  temp=c.split('\t')
  video_names.append(temp[0])
  labels.append(temp[1])
  data_use.append(temp[2])

train_video_names=[video_names[i] for i in range(len(data_use)) if data_use[i] is '1' and int(labels[i]) in range(1, 26)]
train_video_labels=[int(labels[i])-1 for i in range(len(data_use)) if data_use[i] is '1' and int(labels[i]) in range(1, 26)]

test_video_names=[video_names[i] for i in range(len(data_use)) if data_use[i] is '2' and int(labels[i]) in range(1, 26)]
test_video_labels=[int(labels[i])-1 for i in range(len(data_use)) if data_use[i] is '2' and int(labels[i]) in range(1, 26)]

print("Number of training videos:", len(train_video_labels))
print("Number of testing videos:", len(test_video_labels))


Number of training videos: 2409
Number of testing videos: 951
CPU times: user 25.4 ms, sys: 140 µs, total: 25.5 ms
Wall time: 723 ms


In [ ]:
# !ls dataset/images

In [ ]:
%%time
import os
import cv2

videos=[]
for index,vid_folder in enumerate(sorted(os.listdir("dataset/images/"))):
  print(index)
  if index==25:
    break
  # print(vid_folder)
  vid=[]
  for i in os.listdir('dataset/images/'+vid_folder):
    x=cv2.imread('dataset/images/'+vid_folder+'/'+i)
    vid.append(x)
  videos.append(vid)

In [ ]:
print(len(videos))

In [ ]:
#Download pre trained VGG16 model
%%time
import torch
import torch.nn as nn
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

#extract first fully connected layer
first_fully_connected_vgg16 = nn.Sequential(*list(vgg16.classifier.children())[:1])
vgg16.classifier=first_fully_connected_vgg16

#Set up GPU
device = torch.device("cuda:0")
vgg16.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:22<00:00, 24.8MB/s]


CPU times: user 4.8 s, sys: 2.34 s, total: 7.14 s
Wall time: 31.5 s


In [ ]:
%%capture
def crop_2_224x224_and_form_tensors(img):
  normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
  prep = transforms.Compose([ transforms.ToTensor(), normalize ])
  c=[]
  c.append(prep(img[0:224, 0:224]))  
  c.append(prep(img[32:256, 0:224]))
  c.append(prep(img[0:224, 32:256]))
  c.append(prep(img[32:256, 32:256]))
  c.append(prep(img[16:240, 16:240]))
  return c


print("Reading video frames")
# class1_videos=[]

videos=[]
for index,vid_folder in enumerate(sorted(os.listdir("dataset/images/"))):
  print(index)
  if index==3360:
    print(vid_folder)
    break
  vid=[]
  for i in os.listdir('dataset/images/'+vid_folder):
    x=cv2.imread('dataset/images/'+vid_folder+'/'+i)
    vid.append(x)
  videos.append(vid)
# class1_videos=[[cv2.imread('dataset/images/'+vid_folder+'/'+i) for i in os.listdir('dataset/images/'+vid_folder)] for vid_folder in enumerate(os.listdir("dataset/images/"))]
print("Finished reading video frames")


In [ ]:
print(len(videos))

In [ ]:
%%capture
all_ftrs=[]
i=0
print("Generating features for")
for video in videos:
  print("video", i+1, '- dataset/mat/'+video_names[i]+'.mat')
  mean_fetr=None
  video_ftrs=[]
  ftr_data={}
  for frame in video:
    crops=crop_2_224x224_and_form_tensors(frame)
    fetrs=[]
    for crop in crops:
      fetrs.append(vgg16(crop.unsqueeze(0).to(device)))
    fetrs=torch.stack(fetrs)
    mean_fetr=torch.mean(fetrs.cpu(), 0).detach().numpy()[0].tolist()
    video_ftrs.append(mean_fetr)
  ftr_data['Feature']=video_ftrs
  savemat('dataset/mat/'+video_names[i]+'.mat', ftr_data)
  i+=1

In [ ]:
print(len(train_video_names))
print(len(test_video_names))

2409
951


In [ ]:
print("Reading .mat files")
train_video_features=[loadmat('dataset/mat/'+vid) for vid in train_video_names]
print("loaded training features")
test_video_features=[loadmat('dataset/mat/'+vid) for vid in test_video_names]
print("loaded testing features")
print("Finished reading .mat files")

Reading .mat files
loaded training features
loaded testing features
Finished reading .mat files


***
***
## **Problem 2.** Modelling

* ##### **Print the size of your training and test data**

In [ ]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', train_video_features[0]['Feature'].shape)
print('Shape of test/validation data is :', test_video_features[0]['Feature'].shape)

Shape of training data is : (25, 4096)
Shape of test/validation data is : (25, 4096)


In [ ]:
# \*write your codes for modelling using the extracted feature (You can use multiple cells, this is just a place holder)
class ActionRecognition(nn.Module):
  def __init__(self):
    
    super(ActionRecognition, self).__init__()
    
    self.recurrent_layer = torch.nn.LSTM(4096, 1000, 1)
    self.project_layer = torch.nn.Linear(1000, 25)
    
    '''
    #Alternate model
    self.project_layer=torch.nn.Linear(4096, 100)
    self.recurr_layer=torch.nn.LSTM(100, 1000, 1)
    self.dropout_layer=torch.nn.Dropout(p=0.5)
    self.classify=torch.nn.Linear(1000, 15)
    '''
    
  def forward(self, input):
    
    rnn_outputs, (hn, cn) = self.recurrent_layer(input)
    logits = self.project_layer(rnn_outputs[:,-1])
    
    '''
    #Alternate model
    proj=self.project_layer(input)
    rnn_outputs, (hn, cn) = self.recurr_layer(proj)
    drpout=self.dropout_layer(rnn_outputs[:, 1])
    logits = self.classify(drpout)
    '''
    
    return logits

model = ActionRecognition()

optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)
criterion = torch.nn.CrossEntropyLoss()

print(model)

n_epoch=30

for epoch in range(n_epoch):  
  running_loss=0.0
  for i in range(len(train_video_features)):    
    
    #inputs, labels=torch.tensor(train_video_features[i]['Feature']).unsqueeze(0), torch.tensor([train_labels[i]])
    inputs, labels=Variable(torch.tensor(train_video_features[i]['Feature']).unsqueeze(0)), Variable(torch.tensor([train_video_labels[i]]))
    
    # zero the parameter gradients
    optimizer.zero_grad()

    outputs = model(inputs.float())
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    running_loss+=loss.item()
    
  print("epoch:", epoch+1, "----> Average loss:", running_loss/float(len(train_video_features)))
  

ActionRecognition(
  (recurrent_layer): LSTM(4096, 1000)
  (project_layer): Linear(in_features=1000, out_features=25, bias=True)
)
epoch: 1 ----> Average loss: 0.18225095407979702
epoch: 2 ----> Average loss: 0.09481456150268809
epoch: 3 ----> Average loss: 0.057404339215594256
epoch: 4 ----> Average loss: 0.037582505137281515
epoch: 5 ----> Average loss: 0.029437667363568076
epoch: 6 ----> Average loss: 0.025121950728011003
epoch: 7 ----> Average loss: 0.021359364879323443
epoch: 8 ----> Average loss: 0.018146090203477433
epoch: 9 ----> Average loss: 0.01601811330089266
epoch: 10 ----> Average loss: 0.013497788622909802


KeyboardInterrupt: ignored

In [ ]:


correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_video_features)):
        images, labels = torch.tensor(test_video_features[i]['Feature']).unsqueeze(0), torch.tensor([test_video_labels[i]])
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


print("\nAccuracy : "+str(100 * correct / float(total))+"%")


Accuracy : 35.75184016824395%


---
---
## **Problem 3.** Evaluation

In [ ]:
# \*write your codes for evaluation (You can use multiple cells, this is just a place holder)

print("Generating long vectors for SVM\n")

#Stacking up training features
train_long_vectors=[np.asarray(video_ftrs['Feature']).flatten('F') for video_ftrs in train_video_features]

#stacking up testing features
test_long_vectors=[np.asarray(video_ftrs['Feature']).flatten('F') for video_ftrs in test_video_features]

print("starting SVM training")
svm_train_start_time=time.time()
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(train_long_vectors, train_video_labels)
svm_train_end_time=time.time()
print("Total time for training: "+str(((svm_train_end_time-svm_train_start_time))/float(60))+" mins")

print("\nSVM testing")
svm_test_start_time=time.time()
pred = clf.predict(test_long_vectors)
accuracy=np.mean(pred==test_video_labels)*100
svm_test_end_time=time.time()
print("Total time for testing: "+str(svm_test_end_time-svm_test_start_time)+" secs")
print("\nAccuracy : "+str(accuracy)+"%")

* ##### **Print the train and test accuracy of your model** 

In [ ]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f :' %(100.00) )
print('Test accuracy is %2.3f :' %(100.00) )

* ##### **Print the train and test and test accuracy of SVM** 

In [ ]:
# Don't hardcode the train and test accuracy
print('Training accuracy is %2.3f :' %(100.00) )
print('Test accuracy is %2.3f :' %(100.00) )

## **Problem 4.** Report

## **Bonus**


* ##### **Print the size of your training and test data**

In [ ]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [ ]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.